# Steam Games Analysis

In [7]:
from dotenv import load_dotenv
import os
import pymysql
import pandas as pd

load_dotenv()

conn = pymysql.connect(host='localhost',
                        port=int(3306),
                        user='root',
                        passwd=os.getenv('MYSQL_PASSWORD'),
                        db='steam')

games = pd.read_sql_query('''
                    SELECT * 
                    FROM app_id_info
                    LEFT JOIN games_genres
                    ON app_id_info.appid = games_genres.appid                    
                    ''', conn)
games.head()



,appid,Title,Type,Price,Release_Date,Rating,Required_Age,Is_Multiplayer,appid,Genre
0,63942,Men of War: Vietnam Special Edition Upgrade Pack,dlc,2.99,1970-01-01,-1,0,1,63942.0,Strategy
1,414120,Modbox,game,14.99,2016-04-05,-1,0,0,414120.0,Casual
2,414120,Modbox,game,14.99,2016-04-05,-1,0,0,414120.0,Early Access
3,414120,Modbox,game,14.99,2016-04-05,-1,0,0,414120.0,Indie
4,414120,Modbox,game,14.99,2016-04-05,-1,0,0,414120.0,Simulation
